In [1]:
import numpy as np
import glob2 as glob
import pandas as pd
import copy
from tqdm import tqdm

In [2]:
files = glob.glob('128_model_weights_3_layers/*.npy')

weights_dict = {'bias' : {}, 'weight' : {}}

for file in files:
    layer_name = file.split('/')[-1].split('.')[0]
    if layer_name.split('_')[-1] == 'bias':
        weights_dict['bias'].update({layer_name.split('_')[0]: np.load(file)})
    else:
        weights_dict['weight'].update({layer_name.split('_')[0]: np.load(file)})

test_df = pd.read_csv('mnist_test.csv')
order= ['fc1', 'fc2', 'fc3']

In [3]:
def choose_neurons_numeric(chosen_neurons, _weights_dict):
    
    weights_dict_ = copy.deepcopy(_weights_dict)
        
    weights_dict_['weight']['fc1'] = weights_dict_['weight']['fc1'][chosen_neurons[0], :]
    weights_dict_['bias']['fc1'] = weights_dict_['bias']['fc1'][chosen_neurons[0]]

    weights_dict_['weight']['fc2'] = weights_dict_['weight']['fc2'][chosen_neurons[1], :][:, chosen_neurons[0]]
    weights_dict_['bias']['fc2'] = weights_dict_['bias']['fc2'][chosen_neurons[1]]

    weights_dict_['weight']['fc3'] = weights_dict_['weight']['fc3'][: ,chosen_neurons[1]]
    
    return weights_dict_


def choose_neurons_slicing(chosen_neurons, _weights_dict):
    weights_dict_ = copy.deepcopy(_weights_dict)

    #############
    ## LAYER 1 ##
    #############

    # Adding expression to the weights of Layer 1
    print('Adding expression to the weights of Layer 1')
    new_fc = []
    for i in tqdm(range(weights_dict_['weight']['fc1'].shape[0])):
        new_fc_row = []
        for j in range(weights_dict_['weight']['fc1'].shape[1]):
            new_fc_row.append(chosen_neurons[0][i] * weights_dict_['weight']['fc1'][i][j])
        new_fc.append(new_fc_row)

    weights_dict_['weight']['fc1'] = np.array(new_fc)

    # Adding expression to the Bias of Layer 1
    print('Adding expression to the bias of Layer 1')
    new_fc = []
    for i in tqdm(range(weights_dict_['bias']['fc1'].shape[0])):
        new_fc.append(chosen_neurons[0][i] * weights_dict_['bias']['fc1'][i])
    
    weights_dict_['bias']['fc1'] = np.array(new_fc)


    #############
    ## LAYER 2 ##
    #############

    # Adding expression to the weights of Layer 2
    print('Adding expression to the weights of Layer 2')
    
    new_fc = weights_dict_['weight']['fc2'].copy().astype(object)
    for i in tqdm(range(weights_dict_['weight']['fc2'].shape[0])):
        for j in range(weights_dict_['weight']['fc2'].shape[1]):
            new_fc[i][j] *= (chosen_neurons[1][i] * chosen_neurons[0][j])

    weights_dict_['weight']['fc2'] = new_fc

    # Adding expression to the Bias of Layer 2
    print('Adding expression to the bias of Layer 2')
    new_fc = []
    for i in tqdm(range(weights_dict_['bias']['fc2'].shape[0])):
        new_fc.append(chosen_neurons[1][i] * weights_dict_['bias']['fc2'][i])
    
    weights_dict_['bias']['fc2'] = np.array(new_fc)

    #############
    ## LAYER 3 ##
    #############
    
    # Adding expression to the weights of Layer 3
    print('Adding expression to the weights of Layer 3')
    
    new_fc = weights_dict_['weight']['fc3'].copy().astype(object)
    for i in tqdm(range(weights_dict_['weight']['fc3'].shape[0])):
        for j in range(weights_dict_['weight']['fc3'].shape[1]):
            new_fc[i][j] *= chosen_neurons[1][j]

    weights_dict_['weight']['fc3'] = new_fc

    return weights_dict_

def predict_expressions(expression_matrix, weights_dict_):

    x = np.dot(weights_dict_['weight']['fc1'], expression_matrix.T) + weights_dict_['bias']['fc1']
    # x = np.maximum(0, x)

    print(x)
    
    for i in x:
        print(i)

    print(x.shape)

    print('Done 1 layer')

    x = np.dot(weights_dict_['weight']['fc2'], x.T) + weights_dict_['bias']['fc2']
    # x = np.maximum(0, x)


    print('Done 2 layer')

    x = np.dot(weights_dict_['weight']['fc3'], x.T) + weights_dict_['bias']['fc3']
    # x = np.maximum(0, x)

    print('Done 3 layer')

    x /= np.sum(x) # softmax

    return x

def predict_numeric(img_df_row, weights_dict_):

    x = np.dot(weights_dict_['weight']['fc1'], img_df_row.T) + weights_dict_['bias']['fc1']
    x = np.maximum(0, x)

    x = np.dot(weights_dict_['weight']['fc2'], x.T) + weights_dict_['bias']['fc2']
    x = np.maximum(0, x)

    x = np.dot(weights_dict_['weight']['fc3'], x.T) + weights_dict_['bias']['fc3']
    x = np.maximum(0, x)

    x /= np.sum(x) # softmax

    return x

def print_neurons_shape(weights_dict_):
    print('Layer Number\t|\t Weights Shape\t|\t Bias Shape')
    print('-'*60)
    print('Layer 1  \t|\t', weights_dict_['weight']['fc1'].shape, '\t|\t' , weights_dict_['bias']['fc1'].shape)
    print('Layer 2  \t|\t', weights_dict_['weight']['fc2'].shape, '\t|\t' , weights_dict_['bias']['fc2'].shape)
    print('Layer 3  \t|\t', weights_dict_['weight']['fc3'].shape, '\t|\t' , weights_dict_['bias']['fc3'].shape)
    
def cross_entropy_loss(label, y_hat):
    y = np.zeros(10)
    y[label] = 1
    return -np.sum(y * np.log(y_hat))

print_neurons_shape(weights_dict)

Layer Number	|	 Weights Shape	|	 Bias Shape
------------------------------------------------------------
Layer 1  	|	 (128, 784) 	|	 (128,)
Layer 2  	|	 (128, 128) 	|	 (128,)
Layer 3  	|	 (10, 128) 	|	 (10,)


In [4]:
### VALIDATION DRAFT: DO NOT REMOVE AND DO NOT RUN!!! ###

# # chosen_fc1 = np.random.randint(0, 2, 2048)
# # chosen_fc2 = np.random.randint(0, 2, 2048)

# # print(chosen_fc1.sum(), chosen_fc2.sum())

# # expressioned_weights = choose_neurons_expression([chosen_fc1, chosen_fc2], weights_dict)

# # 1012 1032

# print((expressioned_weights['weight']['fc1'].sum(axis = 0) != 0).sum() , (expressioned_weights['weight']['fc1'].sum(axis = 1) != 0).sum())
# print((expressioned_weights['weight']['fc2'].sum(axis = 0) != 0).sum() , (expressioned_weights['weight']['fc2'].sum(axis = 1) != 0).sum())
# print((expressioned_weights['weight']['fc3'].sum(axis = 0) != 0).sum() , (expressioned_weights['weight']['fc3'].sum(axis = 1) != 0).sum())

### DoCplex

In [154]:
from docplex.cp.model import CpoModel as Model

m = Model()

X_fc1 = np.array(m.binary_var_list(128, name='Layer 1 Neurons'))
X_fc2 = np.array(m.binary_var_list(128, name='Layer 2 Neurons'))

In [155]:
W_hat_fc1_weight = X_fc1.reshape((-1, 1))  * weights_dict['weight']['fc1']
W_hat_fc1_bias = weights_dict['bias']['fc1'] * X_fc1

W_hat_fc2_weight = X_fc2.reshape((-1, 1)) * weights_dict['weight']['fc2']
W_hat_fc2_weight = W_hat_fc2_weight * X_fc1.reshape((1, -1))
W_hat_fc2_bias = X_fc2 * weights_dict['bias']['fc2']

W_hat_fc3_weight = weights_dict['weight']['fc3'] * X_fc2.reshape((1, -1))

In [156]:
sub_chunk = 10
data = test_df.drop(columns=['label']).iloc[:sub_chunk]
true_class = test_df.iloc[:sub_chunk]['label']

In [157]:
out_x = np.dot(data, W_hat_fc1_weight.T) + W_hat_fc1_bias
print(out_x.shape)
out_x = np.dot(out_x, W_hat_fc2_weight) + W_hat_fc2_bias
print(out_x.shape)
out_x = np.dot(out_x, W_hat_fc3_weight.T) + weights_dict['bias']['fc3']
print(out_x.shape)

(10, 128)
(10, 128)
(10, 10)


In [158]:
### Find index of max for each item in the batch
maxs_ = [m.binary_var_list(sub_chunk, name='Largest Index in item {i}') for i in range(len(out_x))]
for i in range(sub_chunk):
    m.add_constraint(m.sum(maxs_[i][o] for o in range(10)) == 1)
    for u in range(10):
        m.add_constraint(out_x[i][u] >= m.sum(out_x[i][j] - 10e6*(1-maxs_[i][u]) for j in range(10)))


max_of_each_input = []
for i in range(sub_chunk):
    max_of_each_input.append(m.sum(maxs_[i][j] * out_x[i][j] for j in range(10)))

In [159]:
### Apply softmax on the max of each input

softmax_of_each_input = []
for i in range(sub_chunk):
    softmax_of_each_input.append(np.e ** max_of_each_input[i] / m.sum(np.e ** out_x[i][j] for j in range(10)))

In [160]:
### Calculate Loss for each input

summation_of_loss = []

for i in range(sub_chunk):
    summation_of_loss.append(-1 * m.log(softmax_of_each_input[i]) *  true_class[i])

summation_of_loss = m.sum(summation_of_loss)

In [161]:
number_of_nuerons_used = X_fc1.sum() + X_fc2.sum()

m.minimize(number_of_nuerons_used + summation_of_loss)

In [162]:
m.solve(log_output=True)

KeyboardInterrupt: 